# Código 13: Malhas e Texturas

## Código pré loop principal

### Inicialização do glfw e criação da janela

In [43]:
#Bibliotecas
try:
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image
except ImportError:
    !pip install glfw
    !pip install pyopengl
    !pip install numpy
    !pip install glm
    !pip install pillow
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

#get_dim_pos: retorna tamanho da tela e posição da tela
def get_dim_pos(per_width = 0.6, per_height = 0.6): 
    # Obtendo configurações do monitor
    monitores = glfw.get_monitors()
    monitor = monitores[0]
    video_mode = glfw.get_video_mode(monitor)
    WIDTH_WINDOW, HEIGHT_WINDOW = video_mode.size
    # Definindo proporção que se quer do monitor
    WIDTH_WINDOW : int = int(per_width*WIDTH_WINDOW)
    HEIGHT_WINDOW : int = int(per_height*HEIGHT_WINDOW)
    POSX_WINDOW : int = (video_mode.size[0] - WIDTH_WINDOW) // 2
    POSY_WINDOW : int = (video_mode.size[1] - HEIGHT_WINDOW) // 2
    return WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW

# Pega tamanho da tela e posição da tela
WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW = get_dim_pos(0.6,0.6)
# Criando janela
TITLE: str = "Cilindro"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
glfw.set_window_pos(window, POSX_WINDOW, POSY_WINDOW)
glfw.make_context_current(window)

### Shaders: Vertex e Fragment

No shader de vértice, adicionamos um atributo e um `varying`:
* `attribute vec2 texture_coord`: representa as coordenadas de textura 2D associadas ao vértice
* `varying`: um tipo de qualificador em OpenGL que é dedicado para calcular valores intermediários entre superficies entre primitivas
* `varying vec2 out_texture`: esta variável será preenchida com as coordenadas de textura associadas a cada vértice, porém, agora, de forma interpolada automaticamente entre os vértices ao longo da primitiva durante o processo de renderização. Ou seja, ela receberá, caso seja necessário, a média das texturas dos vértices vizinhos. Essa necessidade pode acontecer quand o objeto for distorcido
* `out_texture = vec2(texture_coord)`: atribuição da interpolação


No shader de fragmento, adicionamos um `uniform` e um `varying`:
* `varying vec2 out_texture`: coordenadas com valores interpolados dos vértices de textura (fluxo de dados é igual para os dois shaders)
* `uniform sampler2D samplerTexture`: 
    * `sampler2D`: variável especial em GLSL usado para representar texturas 2D. Ela não armazena a textura de fato, mas possui um identificador para ela
    * `samplerTexture`: nome da variável 
* `vec4 texture = texture2D(samplerTexture, out_texture)`: função usada para amostrar textura 2D. O primeiro argumento é o identificado e a segunda as coordenadas
* `gl_FragColor = texture`: define o valor final da cor do fragmento como igual a cor da textura

Também aproveitamos para separar as matrizes para serem calculadas na GPU

dúvida: qual necessidade de vec2() no out_texture

dúvida: o fluxo de dados do out_texture é a mesma para os dois shaders?

In [44]:
#GLSL para Vertex Shader
vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_rot_x;
        uniform mat4 mat_rot_y;
        uniform mat4 mat_rot_z; 
        uniform mat4 mat_scale;  
        uniform mat4 mat_transl; 

        attribute vec2 texture_coord;
        varying vec2 out_texture;

        void main(){
            gl_Position = mat_transl * mat_scale * mat_rot_x * mat_rot_y * mat_rot_z * vec4(position,1.0);
            out_texture = vec2(texture_coord);
        }
        """

#GLSL para Fragment Shader
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Solicitando espaço, compilando e linkando

In [45]:
#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

### Extraindo os dados dos arquivos Wavefront

* Arquivo do tipo [Wavefront](https://en.wikipedia.org/wiki/Wavefront_.obj_file): possuem informações para representação de objetos 3D como posição dos vértices de malha e textura. A seguir, a explicação, em ordem de uso dentro do arquivo, do que é cada item
    * `v`: vértices
    * `vt`: vértices de textura
    * `vn`: vértices normal (para iluminação e sombreamento)
    * `vp`: vértices paramétricos (para superfícies continuas e específicas - não usamos)
    * `f`: representa a função de como construir o polígono
        * `a/b/c`: `a` é a posição do vértice, `b` posição do vértice textura e `c` informação sobre iluminação

* Malha: coleção de faces. Cada face é uma coleção de vértices
* Texturização: forma eficiente de lidar com as diferenças de reflectância difusa ponto-a-ponto em uma superfície
    * Obs:  reflectância difusa ponto-a-ponto é uma medida que descreve como a quantidade de luz refletida de uma superfície varia em relação à direção da luz incidente e da direção da observação
    * Cada triângulo da malha é mapeado em um triângulo da textura
* Pixel e Textel:
    * Pixel: representação discreta de cor em uma grade bidimensional
    * Textel: unidade que compõe uma textura aplicada a um objeto 3D
* Mapeamento de texturas: existem três espaços neste contexto. O primeiro é o espaço de textura que possui todas as informações de textura. O segundo é o espaço do objeto o qual importará informações do espaço de textura. O terceiro é o espaço da janela que exibirá o objeto texturizado

![Mapeamento de texturas](13_Malhas_Texturas01.png)
![Mapeamento de texturas](13_Malhas_Texturas02.png)

DUCIDA: EM QUE MOEMENTO SAO USADO  OS OUTROS ELEMTNOS DA FCE?

In [46]:
# Função: carrega o arquivo Wavefront
# Entrada: nome do arquivo
# Saida: estrutura que armazena o elemento (vertices, textura e faces)
def load_model_from_file(file_path_name):
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # Abre o arquivo obj (wavefront) para leitura
    for line in open(file_path_name, "r"): ## para cada linha do arquivo .obj
        # Se for comentário, ignore esta linha e use a próxima
        if line.startswith('#'): continue

        # Quebra a linha por espaço
        values = line.split()
        # Se não há informações na linha, ignore esta linha e use a próxima
        if not values: continue

        # Recupera as informações
        ### Armazena coordenadas dos vertices do elemento no vetor vertices
        if values[0] == 'v':
            vertices.append(values[1:4])
        ### Armazena coordenadas das texturas no vetor texture_coords
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])
        ### Define o material 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        ### Armazena informações sobre a construção das faces
        elif values[0] == 'f':
            face = []
            face_texture = []
            # Para cada elemento da linha que define a função
            for bloco in values[1:]:
                # Separa o elemento em vetor de elementos separando os números que são separados por /
                positions = bloco.split('/')
                # Adiciona o primeiro número na face (que representa o número da linha que encontra-se um vértice para da figura)
                face.append(int(positions[0]))
                # Se o vetor com elementos separados por / for maior ou igual que dois
                # Se o segundo número do elemento for maior do que zero
                if len(positions) >= 2 and len(positions[1]) > 0:
                    # Adicione o segundo número na face de textura (que representa o número da linha que encontra-se um vértice de textura da figura)
                    face_texture.append(int(positions[1]))
                else:
                    # Se não for maior ou igual a dois ou não for maior que zero, coloque zero na textura
                    face_texture.append(0)
            # Após conseguir, provavelmente, os três valores para face, os três valores para textura e o tipo de material, insira na faces
            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces


    return model

### Associando textura a um id (INCOMPLETO)

* `glBindTexture(GL_TEXTURE_2D, texture_id)`: associa a textura identificada por `texture_id` ao contexto OpenGL, especificando que é 2D (`GL_TEXTURE_2D`). Assim, todas as informações nas linhas subsequentes se refere à textura_id
* `glTexParameteri()`: permite a configuração das texturas
    * `glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)`:
    * `glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)`:
    * `glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)`:
    * `glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)`:
* `glTexImage2D()`: carrega os dados da imagem como um textura 2D
    * `GL_TEXTURE_2D`: indica que o tipo de textura é 2D
    * `0`: indica que o nível de detalhe é nível de base
    * `GL_RGB`: textura interna utilizará o padrão RGB
    * `img_width` e `img_height`: indica as dimensões da texura
    * `0`: indica que a textura não tem borda
    * `GL_RGB`: textura está vindo em formato RGB
    * `GL_UNSIGNED_BYTE`: indica que os valores dos texels são representados por bytes sem sinais
    * `image_data`: conjuntos de bytes da imagem

In [47]:
#Função: associa id com a textura
#Entradas: o id que queremos associar e o caminho do arquivo .jpg
#Saida: não possui, apenas associa
def load_texture_from_file(texture_id, img_textura):
    #Definindo o id
    glBindTexture(GL_TEXTURE_2D, texture_id)
    #Alterando configurações paramétricas de textura
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    #Abre imagem
    img = Image.open(img_textura)
    #Captura as dimensões
    img_width = img.size[0]
    img_height = img.size[1]
    #Transforma imagem para um sequência de bytes em formato raw de arquivo
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #Carregando os dados da imagem
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### Pega informações de coordenadas de textura e vertice

In [48]:
# Função: retorna uma lista para coordenadas dos vertices do objeto e outra para as da textura
def get_vertices_textures(model):
    vertices_list = []    
    textures_coord_list = []
    # Para cada um das faces (num_line(v), num_line(vt), material)
    for face in model['faces']:
        # Para cada um dos números que representa a linha do vértice
        for vertice_id in face[0]: # Pega o valor a coordenada do vértice
            vertices_list.append( model['vertices'][vertice_id-1] )
        # Para cada um dos números que representa a linha da coordenada da textura
        for texture_id in face[1]:  # Pega o valor a coordenada da textura
            textures_coord_list.append( model['texture'][texture_id-1] )
    return vertices_list, textures_coord_list

In [49]:
# modelo = load_model_from_file('terreno/terreno2.obj')

# ### inserindo vertices do modelo no vetor de vertices
# print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
# for face in modelo['faces']:
#     for vertice_id in face[0]:
#         vertices_list.append( modelo['vertices'][vertice_id-1] )
#     for texture_id in face[1]:
#         textures_coord_list.append( modelo['texture'][texture_id-1] )
# print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

# ### inserindo coordenadas de textura do modelo no vetor de texturas


# ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
# load_texture_from_file(1,'terreno/pedra.jpg')



# modelo = load_model_from_file('casa/casa.obj')

# ### inserindo vertices do modelo no vetor de vertices
# print('Processando modelo casa.obj. Vertice inicial:',len(vertices_list))
# for face in modelo['faces']:
#     for vertice_id in face[0]:
#         vertices_list.append( modelo['vertices'][vertice_id-1] )
#     for texture_id in face[1]:
#         textures_coord_list.append( modelo['texture'][texture_id-1] )
# print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

# ### inserindo coordenadas de textura do modelo no vetor de texturas


# ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
# load_texture_from_file(1,'casa/casa.jpg')



# modelo = load_model_from_file('monstro/monstro.obj')

# ### inserindo vertices do modelo no vetor de vertices
# print('Processando modelo monstro.obj. Vertice inicial:',len(vertices_list))
# for face in modelo['faces']:
#     for vertice_id in face[0]:
#         vertices_list.append( modelo['vertices'][vertice_id-1] )
#     for texture_id in face[1]:
#         textures_coord_list.append( modelo['texture'][texture_id-1] )
# print('Processando modelo monstro.obj. Vertice final:',len(vertices_list))

# ### inserindo coordenadas de textura do modelo no vetor de texturas


# ### carregando textura equivalente e definindo um id (buffer): use um id por textura!
# load_texture_from_file(3,'monstro/monstro.jpg')

### Fazendo carregamentos

* `glEnable(GL_TEXTURE_2D)`: ativa a funcionalidade de texturas 2D 
* `glGenTextures(num_textures)`: gera IDs para texturas


PAREI AQUI NAS CLASSES 

In [50]:
#Ativando texturas 2D
glEnable(GL_TEXTURE_2D)
#Gerando ids
num_textures = 10
textures = glGenTextures(num_textures)

#Carregando modelo
PATH_WAVE : str = 'objetos_wavefront'
CAIXA_PATH_OBJ : str = f'{PATH_WAVE}\caixa\caixa.obj'
CAIXA_PATH_JPG : str = f'{PATH_WAVE}\caixa\caixa.jpg'
modelo = load_model_from_file(CAIXA_PATH_OBJ)
#Carregando imagem
id_caixa = 0
load_texture_from_file(id_caixa, CAIXA_PATH_JPG)
#Carregando coordenadas de vertice e textura
vertices_list, textures_coord_list = get_vertices_textures(modelo)

### Construção dos vetores de dados

In [51]:
#Finaliza a modelagem dos dados de vértices
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

#Finaliza a modelagem dos dados de texturas
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)])
textures['position'] = textures_coord_list

### Manipulação dos espaços de dados

* Vale ressaltar que na função `glBufferdata`, estamos usando o argumento `GL_STATIC_DRAW` e não `GL_DYNAMIC_DRAW`.
    * `GL_STATIC_DRAW`: otimiza quando os dados do buffer não vão ficar mudando constantemente
    * `GL_DYNAMIC_DRAW`: otimiza quando os dados do buffer vão ficar mudando constantemente
* Outro ponto: agora há dois fluxos de dados (vertices e textura). Logo, precisa-se solicitar dois slots para GPU

In [52]:
#Solicita dois buffers para GPU
buffer = glGenBuffers(2)

# Enviando dados de vértice
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

# Enviando dados de textura
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
#Subindo os dados de textura para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
#Encontrando informações de stride e offset das texturas
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "texture_coord" e habilitando
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
#Linkando dados ao atributo "texture_coord"
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

#Capturando posição dos uniform "mat_rot_x, mat_rot_y, mat_rot_z, mat_scale e mat_transl"
loc_mat_rot_x = glGetUniformLocation(program, "mat_rot_x")
loc_mat_rot_y = glGetUniformLocation(program, "mat_rot_y")
loc_mat_rot_z = glGetUniformLocation(program, "mat_rot_z")
loc_mat_scale = glGetUniformLocation(program, "mat_scale")
loc_mat_transl = glGetUniformLocation(program, "mat_transl")

### Criando funções de impressão de objetos

In [53]:
def desenha_caixa(id_caixa, inicial_vertice, num_vertices):
    # Ativa textura com id id_caixa
    glBindTexture(GL_TEXTURE_2D, id_caixa)
    # Desenha o elemento
    glDrawArrays(GL_TRIANGLES, inicial_vertice, num_vertices)

# def desenha_terreno():
#     # aplica a matriz model
#     # rotacao
#     angle = 0.0;
#     r_x = 0.0; r_y = 0.0; r_z = 1.0;
#     # translacao
#     t_x = 0.0; t_y = -1.01; t_z = 0.0;
#     # escala
#     s_x = 20.0; s_y = 20.0; s_z = 20.0;
#     mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
#     loc_model = glGetUniformLocation(program, "model")
#     glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
#     #define id da textura do modelo
#     glBindTexture(GL_TEXTURE_2D, 1)
#     # desenha o modelo
#     glDrawArrays(GL_TRIANGLES, 36, 42-36) ## renderizando

# def desenha_casa():
#     #define id da textura do modelo
#     glBindTexture(GL_TEXTURE_2D, 1)
#     # desenha o modelo
#     glDrawArrays(GL_TRIANGLES, 36, 1434) ## renderizando

# def desenha_monstro(rotacao_inc):
#     #define id da textura do modelo
#     glBindTexture(GL_TEXTURE_2D, 3)
#     # desenha o modelo
#     glDrawArrays(GL_TRIANGLES, 1476, 7584-1476) ## renderizando

### Eventos de teclado e mouse

In [54]:
polygonal_mode = False

angle = 0.0
scale = 1.0
pos_x = pos_y = pos_z = 0.0

def key_event(window,key,scancode,action,mods):
    global angle, pos_x, pos_y, scale

    angular_speed = 0.139
    if key == 81: angle -= angular_speed #Tecla Q
    if key == 82: angle += angular_speed #Tecla R

    scale_speed = 0.2
    if key == 70: scale += scale_speed #Tecla F
    if key == 71: scale -= scale_speed #Tecla G

    linear_speed = 0.05
    if key == 87: pos_y += linear_speed #Tecla W
    if key == 83: pos_y -= linear_speed #Tecla S
    if key == 65: pos_x -= linear_speed #Tecla A
    if key == 68: pos_x += linear_speed #Tecla D

    # print('[key event] key=',key)
    # print('[key event] scancode=',scancode)
    # print('[key event] action=',action)
    # print('[key event] mods=',mods)
    # print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
    
glfw.set_mouse_button_callback(window,mouse_event)

### Exibindo na tela

In [55]:
glfw.show_window(window)

## Loop principal

#por que em antes estavamos trabalhdno com gl_false e agr n masi


#dúvida porque nao estou conseguindo mover minha caixa para os lados

In [56]:
# Habilita 3D
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(window):
    # Lê eventos
    glfw.poll_events()
    # Limpa buffer de cor e Z-buffer
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    # Define cor como sendo RGB(1.0,1.0,1.0)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    # Define se será mostrado a malha ou não
    if polygonal_mode:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    else:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)

    mat_translation = np.array([    1.0, 0.0, 0.0, pos_x, 
                                    0.0, 1.0, 0.0, pos_y, 
                                    0.0, 0.0, 1.0, pos_z, 
                                    0.0, 0.0, 0.0,   1.0], np.float32)
    
    scale_x = scale_y = scale_z = scale
    mat_scale =      np.array([     scale_x,     0.0,     0.0, 0.0, 
                                        0.0, scale_y,     0.0, 0.0, 
                                        0.0,     0.0, scale_z, 0.0, 
                                        0.0,     0.0,     0.0, 1.0], np.float32)
    
    cos = math.cos(angle)
    sin = math.sin(angle)    
    mat_rotation_z = np.array([     cos, -sin, 0.0, 0.0, 
                                    sin,  cos, 0.0, 0.0, 
                                    0.0,  0.0, 1.0, 0.0, 
                                    0.0,  0.0, 0.0, 1.0], np.float32)
    
    mat_rotation_x = np.array([     1.0, 0.0,  0.0, 0.0, 
                                    0.0, cos, -sin, 0.0, 
                                    0.0, sin,  cos, 0.0, 
                                    0.0, 0.0,  0.0, 1.0], np.float32)
    
    mat_rotation_y = np.array([      cos,  0.0, sin, 0.0, 
                                     0.0,  1.0, 0.0, 0.0, 
                                    -sin,  0.0, cos, 0.0, 
                                     0.0,  0.0, 0.0, 1.0], np.float32)

    glUniformMatrix4fv(loc_mat_rot_x, 1, GL_TRUE, mat_rotation_x) 
    glUniformMatrix4fv(loc_mat_rot_y, 1, GL_TRUE, mat_rotation_y) 
    glUniformMatrix4fv(loc_mat_rot_z, 1, GL_TRUE, mat_rotation_z) 
    glUniformMatrix4fv(loc_mat_scale, 1, GL_TRUE, mat_scale) 
    glUniformMatrix4fv(loc_mat_transl, 1, GL_TRUE, mat_translation) 

    desenha_caixa(id_caixa, 0, 36)   
    # desenha_terreno()
    # desenha_casa()
    # desenha_monstro(rotacao_inc)  
 
    glfw.swap_buffers(window)

glfw.terminate()

[mouse event] button= 0
[mouse event] action= 1
[mouse event] mods= 0
-------
[mouse event] button= 0
[mouse event] action= 0
[mouse event] mods= 0
-------
